In [ ]:
#this can be used to join 2 catalogs for a joint fit. Scaling a column is also included. 

In [166]:
import numpy as np
from astropy.table import Table, join, vstack
#open 2 files to join
file_matcha = ""
file_xmm = ""
catalog_matcha = Table.read(file_matcha)
catalog_xmm = Table.read(file_xmm)


print(f"The number of columns in `catalog_matcha` are {len(catalog_matcha.colnames)}")
print(f"The number of rows in catalog_matcha are {len(catalog_matcha)}.")
print(f"The number of rows are in catalog_xmm are {len(catalog_xmm)}.")

catalog_xmm[0:5]

The number of rows are 161.
The number of rows are 1092.


XCS_NAMES_1,XCS_NAMES_2,col1,Unnamed: 0.1,RA,DEC,Z_1,LAMBDA_CHISQ,LAMBDA_CHISQ_E,R_LAMBDA,MEM_MATCH_ID,RA_CENT,DEC_CENT,ID_CENT,DEC_xmm,DISTANCE,EXP,ID,MEDIAN_EXP,OBSID,RA_xmm,WORST_EXP,WORST_MEDIAN_EXP,RA_ext,DEC_ext,P_DIS,COUNTS,Pk2RM,Pk2RM_kpc,Pk2RM2,Pk2RM2_kpc,RA_pk,DEC_pk,pnsn,pnsnr_alt,kT,kT_m,kT_p,kT_err,Lx,Lx_m,Lx_p,Lx52,Lx52_m,Lx52_p,LAMBDA_CHISQ_E1
str25,str24,int64,int64,float64,float64,float64,float64,float64,float64,int64,str89,str99,str49,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
XMMXCS J000235.8-020227.5,XMMXCSJ000235.8-020227.5,243,243,0.649614166660452,-2.03944174729577,0.39010683,24.808802,2.2613642,0.7566955,13751,"0.649614166660452,0.647402604517322,0.650837133497822,0.655687554778573,0.660709660929633","-2.03944174729577,-2.03197828580954,-2.03145848443447,-2.04139022344033,-2.01694166950346","142501863,142502130,142502090,142502660,142501240",-2.0286111111111103,0.14756604290788675,27762.0146484375,13751,31515.4306640625,653290201,0.796875,26692.9775390625,27069.23095703125,0.64920033,-2.0409846000000003,30.425720743968988,680.99361,0.04458284358593551,14.153533136494243,0.04458284358593551,14.153533136494234,0.6499766469275086,-2.040090510689373,17.222,16.33,2.14291,0.1664257210000002,0.3757072909999999,0.12649458260029586,0.4179751767,0.027991681182347072,0.02890672672743194,0.1734770535,0.007915371603489152,0.010480165454349183,2.2613642
XMMXCS J000400.2-355653.6,XMMXCSJ000400.2-355653.6,248,248,1.00104699302456,-35.9478493979938,0.51813245,27.447912,2.398124,0.77215034,15193,"1.00104699302456,1.00496334329385,1.00006235952776,1.00220768606892,0.991459040649254","-35.9478493979938,-35.9721189810189,-35.9503624329606,-35.9714384684201,-35.9488692491967","142401587,142402656,142401775,142402605,142401731",-35.9550555555556,0.13696826958557765,44896.494140625,15193,46655.4189453125,145020201,0.832083333333333,37725.16796875,39750.76171875,1.0010573,-35.948244,8.83761472448519,1153.8677,0.13721923416480866,51.20854620553784,0.13721923416480866,51.208546205537864,1.002997438367125,-35.94950383602284,22.3303,20.8108,3.42241,0.2941954530000004,0.381135821,0.09866311663418473,1.075204042,0.06172810768921333,0.051566316665137,0.3778939892,0.014460817597852449,0.015481156442907651,2.398124
XMMXCS J001737.5-005234.2,XMMXCSJ001737.5-005234.2,73,73,4.40665775875568,-0.8782889047076321,0.20821463,61.80211600000001,2.7435331,0.9082401,500,"4.40665775875568,4.4036478772821,4.41369820128273,4.43197222196143,4.42113577173195","-0.878288904707632,-0.853717099179728,-0.870237504031457,-0.867241148525203,-0.872233093568332","189078380,189077757,189078763,189078751,189078964",-0.923666666666667,0.07715508173656564,51380.166015625,500,51384.4873046875,403760701,4.34425,33254.130859375,40512.3388671875,4.4063253,-0.87619162,26.028173154787265,12189.1,0.07743152147596767,15.818489529702676,0.07743152147596767,15.818489529702742,4.406493390460635,-0.8795689223325999,55.7922,52.5696,4.17565,0.10585596800000019,0.10677553100000026,0.02546088620933273,1.770299475,0.02862182248558387,0.02736854278286338,0.5694802411,0.005413722467445425,0.005881083050004232,2.7435331
XMMXCS J001938.0+033635.3,XMMXCSJ001938.0-033635.3,9,9,4.91105595050762,3.59932929627596,0.27425575,139.48044,4.365691,1.0688154,17,"4.91105595050762,4.90685473048909,4.90917120339313,4.90346134559408,4.90720740015353","3.59932929627596,3.6302137316629,3.60703825786776,3.62043694963565,3.6294193326428","143999329,143998825,144000405,143999694,143998953",3.62888888888889,0.041495948958367695,57541.447265625,17,66652.734375,693010301,4.881875,56180.8046875,57458.4189453125,4.9083898,3.6098177000000002,162.90251264384614,58596.613,0.3775585070541489,94.73417637364768,0.3775585070541489,94.73417637364797,4.909123355579328,3.6053190510251065,106.246,99.3671,6.79351,0.

In [167]:
#column names must be identical to join the two catalogs with this method. Any "unmatched" columns will not be included in the final joint catalog. 
#make required cuts (instead of using CluStR to make cuts) for flags that are not present in both catalogs.
#An alternative method would be to add columns to the catalogs to have all matching flags, then run it through CluStR. 

#General flags used.

catalog_matcha2 = catalog_matcha[catalog_matcha['bad_mode'] == False]
#CluStR will automatically delete nans, but it can be useful to do before so you know how many clusters come from each catalog
catalog_matcha3 = catalog_matcha2[~np.isnan(catalog_matcha2["r2500_temperature_scaled"])]
catalog_matcha4 = catalog_matcha3[catalog_matcha3['edge_r2500'] == False]
catalog_matcha5 = catalog_matcha4[catalog_matcha4['edge_bkgd'] == False]
catalog_matcha6 = catalog_matcha5[catalog_matcha5['masked'] == False]
catalog_matcha7 = catalog_matcha6[catalog_matcha6['overlap_r500'] == False]
catalog_matcha8 = catalog_matcha7[catalog_matcha7['overlap_r2500'] == False]
catalog_matcha9 = catalog_matcha8[catalog_matcha8['overlap_bkgd'] == False]
catalog_matcha10 = catalog_matcha9[catalog_matcha9['500_kiloparsecs_SNR'] > 9.0]
catalog_matcha11 = catalog_matcha10[catalog_matcha10['Redshift'] > 0.2]
catalog_matcha12 = catalog_matcha11[catalog_matcha11['Redshift'] < 0.65]
#don't forget to delete repeat clusters. 
#Sometimes the catalogs don't have the same column name for MEM_MATCH_ID, so the column name in one of the catalogs needs to be renamed first to match.
#Edit to rename if needed: 
#catalog_xmm['MEM'].name = "MEM_MATCH_ID"
#make a column for repeats. 
catalog_matcha['repeat'] = np.isin(catalog_matcha['MEM_MATCH_ID'], catalog_xmm['MEM_MATCH_ID'])
#delete repeats
catalog_matcha13 = catalog_matcha12[catalog_matcha12['repeat'] == False]

#amount of clusters left after cuts. Make sure to use new catalog name that with the cut clusters. 
print(f"The number of columns are {len(catalog_matcha13.colnames)}")
print(f"The number of rows are {len(catalog_matcha13)}.")
catalog_matcha13[0:3]

#repeat this for second catalog if it needs any cuts


The number of columns are 124
The number of rows are 64.


XCS_NAMES_1,XCS_NAMES_2,col1,Unnamed: 0.1,RA,DEC,Z_1,LAMBDA_CHISQ,LAMBDA_CHISQ_E,R_LAMBDA,MEM_MATCH_ID,RA_CENT,DEC_CENT,ID_CENT,DEC_xmm,DISTANCE,EXP,ID,MEDIAN_EXP,OBSID,RA_xmm,WORST_EXP,WORST_MEDIAN_EXP,RA_ext,DEC_ext,P_DIS,COUNTS,Pk2RM,Pk2RM_kpc,Pk2RM2,Pk2RM2_kpc,RA_pk,DEC_pk,pnsn,pnsnr_alt,kT,kT_m,kT_p,kT_err,Lx,Lx_m,Lx_p,Lx52,Lx52_m,Lx52_p,LAMBDA_CHISQ_E1
str25,str24,int64,int64,float64,float64,float64,float64,float64,float64,int64,str89,str99,str49,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
XMMXCS J000235.8-020227.5,XMMXCSJ000235.8-020227.5,243,243,0.649614166660452,-2.03944174729577,0.39010683,24.808802,2.2613642,0.7566955,13751,"0.649614166660452,0.647402604517322,0.650837133497822,0.655687554778573,0.660709660929633","-2.03944174729577,-2.03197828580954,-2.03145848443447,-2.04139022344033,-2.01694166950346","142501863,142502130,142502090,142502660,142501240",-2.0286111111111103,0.14756604290788675,27762.0146484375,13751,31515.4306640625,653290201,0.796875,26692.9775390625,27069.23095703125,0.64920033,-2.0409846000000003,30.425720743968988,680.99361,0.04458284358593551,14.153533136494243,0.04458284358593551,14.153533136494234,0.6499766469275086,-2.040090510689373,17.222,16.33,2.14291,0.1664257210000002,0.3757072909999999,0.12649458260029586,0.4179751767,0.027991681182347072,0.02890672672743194,0.1734770535,0.007915371603489152,0.010480165454349183,2.2613642
XMMXCS J000400.2-355653.6,XMMXCSJ000400.2-355653.6,248,248,1.00104699302456,-35.9478493979938,0.51813245,27.447912,2.398124,0.77215034,15193,"1.00104699302456,1.00496334329385,1.00006235952776,1.00220768606892,0.991459040649254","-35.9478493979938,-35.9721189810189,-35.9503624329606,-35.9714384684201,-35.9488692491967","142401587,142402656,142401775,142402605,142401731",-35.9550555555556,0.13696826958557765,44896.494140625,15193,46655.4189453125,145020201,0.832083333333333,37725.16796875,39750.76171875,1.0010573,-35.948244,8.83761472448519,1153.8677,0.13721923416480866,51.20854620553784,0.13721923416480866,51.208546205537864,1.002997438367125,-35.94950383602284,22.3303,20.8108,3.42241,0.2941954530000004,0.381135821,0.09866311663418473,1.075204042,0.06172810768921333,0.051566316665137,0.3778939892,0.014460817597852449,0.015481156442907651,2.398124
XMMXCS J001737.5-005234.2,XMMXCSJ001737.5-005234.2,73,73,4.40665775875568,-0.8782889047076321,0.20821463,61.80211600000001,2.7435331,0.9082401,500,"4.40665775875568,4.4036478772821,4.41369820128273,4.43197222196143,4.42113577173195","-0.878288904707632,-0.853717099179728,-0.870237504031457,-0.867241148525203,-0.872233093568332","189078380,189077757,189078763,189078751,189078964",-0.923666666666667,0.07715508173656564,51380.166015625,500,51384.4873046875,403760701,4.34425,33254.130859375,40512.3388671875,4.4063253,-0.87619162,26.028173154787265,12189.1,0.07743152147596767,15.818489529702676,0.07743152147596767,15.818489529702742,4.406493390460635,-0.8795689223325999,55.7922,52.5696,4.17565,0.10585596800000019,0.10677553100000026,0.02546088620933273,1.770299475,0.02862182248558387,0.02736854278286338,0.5694802411,0.005413722467445425,0.005881083050004232,2.7435331
XMMXCS J001938.0+033635.3,XMMXCSJ001938.0-033635.3,9,9,4.91105595050762,3.59932929627596,0.27425575,139.48044,4.365691,1.0688154,17,"4.91105595050762,4.90685473048909,4.90917120339313,4.90346134559408,4.90720740015353","3.59932929627596,3.6302137316629,3.60703825786776,3.62043694963565,3.6294193326428","143999329,143998825,144000405,143999694,143998953",3.62888888888889,0.041495948958367695,57541.447265625,17,66652.734375,693010301,4.881875,56180.8046875,57458.4189453125,4.9083898,3.6098177000000002,162.90251264384614,58596.613,0.3775585070541489,94.73417637364768,0.3775585070541489,94.73417637364797,4.909123355579328,3.6053190510251065,106.246,99.3671,6.79351,0.

In [168]:
catalog_matcha13[0:5]

Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes,isin_Vol_Lim,Tx_r2500_in_xmm,Tx_r500_in_xmm,lambda_xray,lambda_xray_err,err_upper_lim,err_lower_lim,r500_temperature_scaled,r2500_temperature_scaled,err_lower_lim_scaled,err_upper_lim_scaled,r2500_temperature_err_low_scaled,r2500_temperature_err_high_scaled,in_xmm
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool
catalogue_5,6,True,True,62.795693300108,-48.3276790985298,0.41268131136894226,"13396, 16355, 17536",178.045,5.02932,5.02932,nan,nan,3.4004e+44,4.21e+42,4.2213e+42,62.81658,-48.313133,62.810727,-48.320797,7.0575,0.33725,0.34416,3.478e+44,6.3701e+42,6.4097e+42,1.1605e+45,1.992e+43,2.0003e+43,500.0,0.0,0.0,78.698,1.0662,1.0662,62.810923,-48.320617,7.1066,0.31181,0.31834,3.5549e+44,6.1126e+42,6.1496e+42,1.1905e+45,1.8811e+43,1.8914e+43,478.99,12.956,12.943,84.785,1.0874,1.0874,62.811077,-48.32109,nan,nan,nan,4.4344e+44,7.9216e+42,8.0348e+42,1.4929e+45,2.8495e+43,2.8093e+43,1085.

In [ ]:
#for when one of the catalogs needs something to be scaled, otherwise skip. 
#if using CluStR to complete the scaling relation, remember to set the pivot point to 1 and change where ln is used to log 10

#from the form: log10(Tx_chandra) = a*log10(Tx_xmm) + b. 
#some algebra:
#10^(log10(Tx_xmm) = 10^((log10(Tx_chandra) - b)/a)
#Tx_xmm = 10^((log10(Tx_Chandra) – b)/a))
#final relation:
#Tx_xmm = Tx Chandra Scaled = 10^((log10(TxChandra) – b)/a))... and here it is: 

#catalog_chandra["r2500_temperature_scaled"] = 10**(((np.log10(catalog_chandra["r2500_temperature"])-b)/a))

#there are alternative ways to do this scaling relation. xmm was chosen as the y value (even though its more algebra), to compare to past papers where xmm was scaled instead of chandra. 

In [169]:
#choose a naming convention from one of the catalogs and rename all of the columns to match. All columns you want included in the final catalog need to have the same name in each catalog. 
catalog_xmm[0:5]
catalog_xmm.rename_column('kT','r2500_temperature')
catalog_xmm['Z_1'].name = "Redshift"
catalog_xmm['LAMBDA_CHISQ'].name = "lambda"
catalog_xmm['LAMBDA_CHISQ_E'].name = "lambda_err_low"
catalog_xmm['LAMBDA_CHISQ_E1'].name = "lambda_err_high"
catalog_xmm['kT_p'].name = "r2500_temperature_err_high_scaled"
catalog_xmm['kT_m'].name = "r2500_temperature_err_low_scaled"
catalog_xmm[0:5]

XCS_NAMES_1,XCS_NAMES_2,col1,Unnamed: 0.1,RA,DEC,Redshift,lambda,lambda_err_low,R_LAMBDA,MEM_MATCH_ID,RA_CENT,DEC_CENT,ID_CENT,DEC_xmm,DISTANCE,EXP,ID,MEDIAN_EXP,OBSID,RA_xmm,WORST_EXP,WORST_MEDIAN_EXP,RA_ext,DEC_ext,P_DIS,COUNTS,Pk2RM,Pk2RM_kpc,Pk2RM2,Pk2RM2_kpc,RA_pk,DEC_pk,pnsn,pnsnr_alt,r2500_temperature_scaled,r2500_temperature_err_low_scaled,r2500_temperature_err_high_scaled,kT_err,Lx,Lx_m,Lx_p,Lx52,Lx52_m,Lx52_p,lambda_err_high
str25,str24,int64,int64,float64,float64,float64,float64,float64,float64,int64,str89,str99,str49,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
XMMXCS J000235.8-020227.5,XMMXCSJ000235.8-020227.5,243,243,0.649614166660452,-2.03944174729577,0.39010683,24.808802,2.2613642,0.7566955,13751,"0.649614166660452,0.647402604517322,0.650837133497822,0.655687554778573,0.660709660929633","-2.03944174729577,-2.03197828580954,-2.03145848443447,-2.04139022344033,-2.01694166950346","142501863,142502130,142502090,142502660,142501240",-2.0286111111111103,0.14756604290788675,27762.0146484375,13751,31515.4306640625,653290201,0.796875,26692.9775390625,27069.23095703125,0.64920033,-2.0409846000000003,30.425720743968988,680.99361,0.04458284358593551,14.153533136494243,0.04458284358593551,14.153533136494234,0.6499766469275086,-2.040090510689373,17.222,16.33,2.14291,0.1664257210000002,0.3757072909999999,0.12649458260029586,0.4179751767,0.027991681182347072,0.02890672672743194,0.1734770535,0.007915371603489152,0.010480165454349183,2.2613642
XMMXCS J000400.2-355653.6,XMMXCSJ000400.2-355653.6,248,248,1.00104699302456,-35.9478493979938,0.51813245,27.447912,2.398124,0.77215034,15193,"1.00104699302456,1.00496334329385,1.00006235952776,1.00220768606892,0.991459040649254","-35.9478493979938,-35.9721189810189,-35.9503624329606,-35.9714384684201,-35.9488692491967","142401587,142402656,142401775,142402605,142401731",-35.9550555555556,0.13696826958557765,44896.494140625,15193,46655.4189453125,145020201,0.832083333333333,37725.16796875,39750.76171875,1.0010573,-35.948244,8.83761472448519,1153.8677,0.13721923416480866,51.20854620553784,0.13721923416480866,51.208546205537864,1.002997438367125,-35.94950383602284,22.3303,20.8108,3.42241,0.2941954530000004,0.381135821,0.09866311663418473,1.075204042,0.06172810768921333,0.051566316665137,0.3778939892,0.014460817597852449,0.015481156442907651,2.398124
XMMXCS J001737.5-005234.2,XMMXCSJ001737.5-005234.2,73,73,4.40665775875568,-0.8782889047076321,0.20821463,61.80211600000001,2.7435331,0.9082401,500,"4.40665775875568,4.4036478772821,4.41369820128273,4.43197222196143,4.42113577173195","-0.878288904707632,-0.853717099179728,-0.870237504031457,-0.867241148525203,-0.872233093568332","189078380,189077757,189078763,189078751,189078964",-0.923666666666667,0.07715508173656564,51380.166015625,500,51384.4873046875,403760701,4.34425,33254.130859375,40512.3388671875,4.4063253,-0.87619162,26.028173154787265,12189.1,0.07743152147596767,15.818489529702676,0.07743152147596767,15.818489529702742,4.406493390460635,-0.8795689223325999,55.7922,52.5696,4.17565,0.10585596800000019,0.10677553100000026,0.02546088620933273,1.770299475,0.02862182248558387,0.02736854278286338,0.5694802411,0.005413722467445425,0.005881083050004232,2.7435331
XMMXCS J001938.0+033635.3,XMMXCSJ001938.0-033635.3,9,9,4.91105595050762,3.59932929627596,0.27425575,139.48044,4.365691,1.0688154,17,"4.91105595050762,4.90685473048909,4.90917120339313,4.90346134559408,4.90720740015353","3.59932929627596,3.6302137316629,3.60703825786776,3.62043694963565,3.6294193326428","143999329,143998825,144000405,143999694,143998953",3.62888888888889,0.041495948958367695,57541.447265625,17,66652.734375,693010301,4.881875,56180.8046875,57458.4189453125,4.9083898,3.6098177000000002,162.90251264384614,58596.613,0.3775585070541489,94.73417637364768,0.3775585070541489,94.

In [171]:
#create an empty table. 

catalog_xmm_matcha = Table()

#use vstack to combine the catalogs. 
catalog_xmm_matcha = vstack([catalog_xmm, catalog_matcha13],join_type='inner')

print(f"The number of columns for `catalog_matcha_detected` are {len(catalog_xmm_matcha.colnames)}")
print(f"The number of rows are {len(catalog_xmm_matcha)}.")

#display table
catalog_xmm_matcha[0:20]


The number of columns for `catalog_matcha_detected` are 8
The number of rows are 225.
The number of rows are 225.


Redshift,lambda,lambda_err_low,MEM_MATCH_ID,r2500_temperature_scaled,r2500_temperature_err_low_scaled,r2500_temperature_err_high_scaled,lambda_err_high
float64,float64,float64,int64,float64,float64,float64,float64
0.39010683,24.808802,2.2613642,13751,2.14291,0.1664257210000002,0.3757072909999999,2.2613642
0.51813245,27.447912,2.398124,15193,3.42241,0.2941954530000004,0.381135821,2.398124
0.20821463,61.80211600000001,2.7435331,500,4.17565,0.10585596800000019,0.10677553100000026,2.7435331
0.27425575,139.48044,4.365691,17,6.79351,0.0854978880000008,0.15795316999999987,4.365691
0.39933878,100.22939,5.156016,225,6.15498,0.4393842809999997,0.44781691000000023,5.156016


In [174]:
#save catalog. 
#if doing csv instead, remember to change the format to "csc"
catalog_xmm_matcha.write("matcha_xmm_joint_scaled_temp_err_no_repeat_r2500_v2.fits", format="fits")

In [173]:
catalog_xmm_matcha[0:20]

Redshift,lambda,lambda_err_low,MEM_MATCH_ID,r2500_temperature_scaled,r2500_temperature_err_low_scaled,r2500_temperature_err_high_scaled,lambda_err_high
float64,float64,float64,int64,float64,float64,float64,float64
0.39010683,24.808802,2.2613642,13751,2.14291,0.1664257210000002,0.3757072909999999,2.2613642
0.51813245,27.447912,2.398124,15193,3.42241,0.2941954530000004,0.381135821,2.398124
0.20821463,61.80211600000001,2.7435331,500,4.17565,0.10585596800000019,0.10677553100000026,2.7435331
0.27425575,139.48044,4.365691,17,6.79351,0.0854978880000008,0.15795316999999987,4.365691
0.39933878,100.22939,5.156016,225,6.15498,0.4393842809999997,0.44781691000000023,5.156016
0.3838058,98.780174,3.8079417,166,5.89563,0.2946439009999997,0.29944346900000074,3.8079417
0.62684613,20.873018,2.2952025,37794,2.6213,0.3501865820000001,0.34026748799999984,2.2952025
0.40607405,25.281706,2.3174372,19762,2.85735,0.35722818899999975,0.6033601540000002,2.3174372
0.55846506,25.083391,2.8836865,9805,1.4697,0.28926382900000003,0.8466204820000001,2.8836865
